In [1]:
import pandas as pd
import numpy as np
import multiprocessing
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
import gc
from time import time
import datetime
from tqdm import tqdm_notebook
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.metrics import roc_auc_score
warnings.simplefilter('ignore')
sns.set()
%matplotlib inline

In [2]:
#  import sys
# !{sys.executable} -m pip install eli5

In [3]:
import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold,TimeSeriesSplit, GroupKFold
from sklearn.model_selection import train_test_split
from sklearn import linear_model
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR

from sklearn.metrics import mean_absolute_error as sk_mean_absolute_error
from sklearn.metrics import roc_auc_score
import numpy as np
import pandas as pd

import eli5
from eli5.sklearn import PermutationImportance

class processutil:
    def _str2class(s):
        if s in globals() and isinstance(globals()[s], type):
                return globals()[s]
        if isinstance(eval(s), type):
            return eval(s)
        if callable(eval(s)):
            return eval(s)
        return None
    

Using TensorFlow backend.


In [4]:
folder_path = '../data/IEEE-CIS-Fraud-Detection/'

In [24]:
# df_train = pd.read_pickle(f'{folder_path}/df_train3.gzde', compression='gzip')#.iloc[:100000,:]
# df_test = pd.read_pickle(f'{folder_path}/df_test3.gzde', compression='gzip')#.iloc[:10000,:]
df_train = df_train.replace([np.inf, -np.inf], np.nan).fillna(-999)
# df_test = df_test.replace([np.inf, -np.inf], np.nan).fillna(-999)

In [6]:
# df_train2 = pd.read_pickle(f'{folder_path}/df_train3.gzde', compression='gzip')#.iloc[:100000,:]
# df_test2 = pd.read_pickle(f'{folder_path}/df_test3.gzde', compression='gzip')#.iloc[:10000,:]

In [6]:
df_oof_train = pd.DataFrame()
df_oof_test = pd.DataFrame()

In [7]:
df_oof_vae_train = pd.read_pickle(f'{folder_path}/oof_vae_train', compression='gzip')
df_oof_vae_test = pd.read_pickle(f'{folder_path}/oof_vae_test', compression='gzip')
df_oof_train = df_oof_vae_train
df_oof_test = df_oof_vae_test

In [8]:
df_oof_lgbm_train = pd.read_pickle(f'{folder_path}/oof_lgbm_train', compression='gzip')
df_oof_lgbm_test = pd.read_pickle(f'{folder_path}/oof_lgbm_test', compression='gzip')
df_oof_lgbm_test['predict'] = df_oof_lgbm_test[np.arange(8)].mean(axis=1)
df_oof_train = pd.merge(df_oof_train, df_oof_lgbm_train[['TransactionID', 'predict']].rename(columns={'predict':'oof_lgbm'}), how='left', on='TransactionID')
df_oof_test = pd.merge(df_oof_test, df_oof_lgbm_test[['TransactionID', 'predict']].rename(columns={'predict':'oof_lgbm'}), how='left', on='TransactionID')

In [9]:
df_oof_xgbm_train = pd.read_pickle(f'{folder_path}/oof_xgbm_train', compression='gzip')
df_oof_xgbm_test = pd.read_pickle(f'{folder_path}/oof_xgbm_test', compression='gzip')
df_oof_xgbm_test['predict'] = df_oof_xgbm_test[np.arange(8)].mean(axis=1)
df_oof_train = pd.merge(df_oof_train, df_oof_xgbm_train[['TransactionID', 'predict']].rename(columns={'predict':'oof_xgbm'}), how='left', on='TransactionID')
df_oof_test = pd.merge(df_oof_test, df_oof_xgbm_test[['TransactionID', 'predict']].rename(columns={'predict':'oof_xgbm'}), how='left', on='TransactionID')

In [10]:
df_oof_mlp_train = pd.read_pickle(f'{folder_path}/oof_mlp_train', compression='gzip')
df_oof_mlp_test = pd.read_pickle(f'{folder_path}/oof_mlp_test', compression='gzip')
df_oof_mlp_test['predict'] = df_oof_mlp_test[np.arange(8)].mean(axis=1)
df_oof_train = pd.merge(df_oof_train, df_oof_mlp_train[['TransactionID', 'predict']].rename(columns={'predict':'oof_mlp'}), how='left', on='TransactionID')
df_oof_test = pd.merge(df_oof_test, df_oof_mlp_test[['TransactionID', 'predict']].rename(columns={'predict':'oof_mlp'}), how='left', on='TransactionID')

In [14]:
df_oof_1dcnn_train = pd.read_pickle(f'{folder_path}/oof_1dcnn_train', compression='gzip')
df_oof_1dcnn_test = pd.read_pickle(f'{folder_path}/oof_1dcnn_test', compression='gzip')
# df_oof_1dcnn_test['predict'] = df_oof_1dcnn_test[np.arange(8)].mean(axis=1)
df_oof_train = pd.merge(df_oof_train, df_oof_1dcnn_train[['TransactionID', 'isFraud']].rename(columns={'isFraud':'oof_1dcnn'}), how='left', on='TransactionID')
df_oof_test = pd.merge(df_oof_test, df_oof_1dcnn_test[['TransactionID', 'isFraud']].rename(columns={'isFraud':'oof_1dcnn'}), how='left', on='TransactionID')

In [15]:
df_oof_train.head()

,loss,kle,reconstructione,mse,TransactionID,oof_lgbm,oof_xgbm,oof_mlp,oof_1dcnn
0,87.781677,-0.000000,87.781677,0.237053,2987000,0.005126,0.007788,0.015279,0.005277
1,75.931198,0.007184,75.924011,0.180677,2987001,0.002398,0.010556,0.034451,0.028677
2,79.129272,-0.000000,79.129272,0.156918,2987002,0.001919,0.012954,0.002952,0.004419
3,83.263702,0.000120,83.263580,0.191927,2987003,0.000922,0.002685,0.000013,0.000802
4,78.967186,0.007210,78.959976,0.201674,2987004,0.001328,0.001727,0.012591,0.000341


In [16]:
df_oof_test.head()

,TransactionID,loss,kle,reconstructione,mse,oof_lgbm,oof_xgbm,oof_mlp,oof_1dcnn
0,3663549,76.213234,0.131185,76.082048,0.170198,0.000186,0.000334,0.000497,0.000324
1,3663550,83.605424,0.125992,83.479432,0.199555,0.001328,0.002672,0.000377,0.002861
2,3663551,80.684792,0.125003,80.559789,0.155993,0.001123,0.000482,0.001226,0.002663
3,3663552,80.136013,0.127257,80.008757,0.156172,0.000416,0.001683,0.004226,0.001190
4,3663553,78.773938,0.126601,78.647337,0.149714,0.000441,0.000414,0.011174,0.003880


In [17]:
df_train = pd.merge(df_train, df_oof_train, how='left', on='TransactionID')
df_test = pd.merge(df_test, df_oof_test, how='left', on='TransactionID')

In [18]:
len(df_test.columns)

933

In [19]:
params = {'num_leaves': 491,
          'min_child_weight': 0.03454472573214212,
          'feature_fraction': 0.3797454081646243,
          'bagging_fraction': 0.4181193142567742,
          'min_data_in_leaf': 106,
          'objective': 'binary',
          'max_depth': -1,
          'learning_rate': 0.006883242363721497,
          "boosting_type": "gbdt",
          "bagging_seed": 11,
          "metric": 'auc',
          "verbosity": -1,
          'reg_alpha': 0.3899927210061127,
          'reg_lambda': 0.6485237330340494,
          'random_state': 47
         }

In [20]:
def process(folds, df_train,df_test, columns, is_output_feature_importance=1, verbose=0):

#     aucs = list()
    his = []
    training_start_time = time()
    df_feature_importances_i_list = []
    df_valid_pred = pd.DataFrame()
    df_test_pred = pd.DataFrame()
    if type(df_test) != type(None):
        df_test_pred['TransactionID'] = df_test['TransactionID']
    
    X,y = df_train.sort_values('TransactionDT')[columns], df_train.sort_values('TransactionDT')['isFraud']
    if type(df_test) != type(None):
        X_test = df_test.sort_values('TransactionDT')[columns]
        
    for fold, (trn_idx, test_idx) in enumerate(folds.split(X, y)):
        start_time = time()
        if verbose > 1:
            print('Training on fold {}'.format(fold + 1))
            
        trn_data = lgb.Dataset(X.iloc[trn_idx], label=y.iloc[trn_idx])
        val_data = lgb.Dataset(X.iloc[test_idx], label=y.iloc[test_idx])
        clf = lgb.train(params, trn_data, 10000, valid_sets = [trn_data, val_data], verbose_eval=1000, early_stopping_rounds=500)
        
        y_trn_pred = clf.predict(X.iloc[trn_idx].values)
        y_val_pred = clf.predict(X.iloc[test_idx].values)
        
        original_index = df_train['TransactionID'].values[test_idx]
        df_valid_pred_i = pd.DataFrame({'TransactionID': original_index, 'predict': y_val_pred, 'fold': np.zeros(y_val_pred.shape[0]) + fold})
        df_valid_pred = pd.concat([df_valid_pred, df_valid_pred_i], axis=0)
        
        y_test_pred = None
        if type(df_test)!=type(None):
            y_test_pred = clf.predict(X_test.values)
            df_test_pred_i = pd.DataFrame({fold: y_test_pred})
            df_test_pred = pd.concat([df_test_pred, df_test_pred_i], axis=1)
        
        trn_auc = roc_auc_score(y.iloc[trn_idx].values, y_trn_pred)
        val_auc = roc_auc_score(y.iloc[test_idx].values, y_val_pred)
        
        his.append({'val_auc':val_auc, 'trn_auc':trn_auc, 'y_val_pred':y_val_pred, 'y_test_pred':y_test_pred, 'test_idx':test_idx})
        
        if is_output_feature_importance:
            best_iter = clf.best_iteration
            clf = lgb.LGBMClassifier(**params, num_boost_round=best_iter)
            clf.fit(X.iloc[trn_idx].values, y.iloc[trn_idx].values)
            perm = PermutationImportance(clf, random_state=42).fit(X.iloc[test_idx].values, y.iloc[test_idx].values)
            df_feature_importances_i2 = eli5.explain_weights_dfs(perm, feature_names=columns, top=len(columns))['feature_importances']
            df_feature_importances_i2 = df_feature_importances_i2.sort_values(by=['feature'])
            df_feature_importances_i2 = df_feature_importances_i2.reset_index(drop=True)
            df_feature_importances_i_list.append(df_feature_importances_i2)
        
#         aucs.append(clf.best_score['valid_1']['auc'])
#         his.append({'val_auc':val_auc, 'trn_auc':trn_auc, 'y_val_pred':y_val_pred, 'y_test_pred':y_test_pred, 'test_idx':test_idx})
        if verbose > 0:
            print('Fold {} finished in {}'.format(fold + 1, str(datetime.timedelta(seconds=time() - start_time))))
    
    his = pd.DataFrame(his)
    
    df_feature_importances = None
    if is_output_feature_importance:
        df_feature_importances = df_feature_importances_i_list[0]
        for idx, df_feature_importances_i in enumerate(df_feature_importances_i_list[1:]):
            df_feature_importances = pd.merge(df_feature_importances, df_feature_importances_i, on='feature', suffixes=('', idx + 1))
            
    df_valid_pred = df_valid_pred.sort_values(by=['TransactionID'])
    df_valid_pred = df_valid_pred.reset_index(drop=True)

    if type(df_test) != type(None):
        df_test_pred = df_test_pred.sort_values(by=['TransactionID'])
        df_test_pred = df_test_pred.reset_index(drop=True)
    
    if verbose > 0:
        print('-' * 30)
        print('Training has finished.')
        print('Total training time is {}'.format(str(datetime.timedelta(seconds=time() - training_start_time))))
        print('Mean AUC:', his.val_auc.mean(), his.trn_auc.mean())
        print('-' * 30)
    return his, df_feature_importances, df_valid_pred, df_test_pred, his.val_auc.mean()

In [21]:
columns = df_test.columns.tolist()

In [22]:
folds = TimeSeriesSplit(n_splits=5)

In [ ]:
his, df_feature_importances, df_valid_pred, df_test_pred, val_metric = process(folds, df_train.iloc[100000:,:],None, columns, is_output_feature_importance=1, verbose=2)

Training on fold 1
Training until validation scores don't improve for 500 rounds.
Early stopping, best iteration is:
[96]	training's auc: 0.986899	valid_1's auc: 0.953091
Fold 1 finished in 0:17:09.928580
Training on fold 2
Training until validation scores don't improve for 500 rounds.
[1000]	training's auc: 0.999997	valid_1's auc: 0.941207
Early stopping, best iteration is:
[967]	training's auc: 0.999996	valid_1's auc: 0.941251


In [26]:
def sort_feature_importances(df_feature_importances, key='average_permutation_weight'):
    df_feature_importances['average_permutation_weight'] = df_feature_importances[
        [col for col in df_feature_importances.columns.tolist() if ('weight' in col) & ('model' not in col)]].mean(
        axis=1)
    df_feature_importances = df_feature_importances.sort_values(by=[key], ascending=False)
    sorted_columns = df_feature_importances.feature.tolist()
    return sorted_columns



In [27]:
sorted_columns = sort_feature_importances(df_feature_importances)

sorted_columns

NameError: name 'df_feature_importances' is not defined

In [ ]:
df_feature_importances

In [29]:
df_test_pred['isFraud']=df_test_pred[np.arange(8)].mean(axis=1)

In [31]:
df_test_pred[['TransactionID','isFraud']].to_pickle(f'{folder_path}/oof_stacking_lgbm_kf8', compression='gzip')